<a href="https://colab.research.google.com/github/sunilchinnahalli/bot-or-not/blob/master/teeetlevel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import re

from keras.layers import Embedding, Dense, LSTM, Dense, Input, concatenate
from keras.models import Model
from keras.utils import plot_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [4]:
#bot_tweets = pd.concat([pd.read_csv('/content/drive/My Drive/twitter/social_spambots_1Tw.csv'), pd.read_csv('/content/drive/My Drive/twitter/social_spambots_2Tw.csv', encoding = "ISO-8859-1"), pd.read_csv('/content/drive/My Drive/twitter/social_spambots_3Tw.csv')]).reset_index(drop=True)

bot_tweets = pd.concat([pd.read_csv('/content/drive/My Drive/twitter/social_spambots_1Tw.csv',nrows=200000)]).reset_index(drop=True)

clean_tweets = pd.read_csv('/content/drive/My Drive/twitter/geniune_accountsTw.csv', nrows=200000)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
requiredColumns = ['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions']
bot_accounts = bot_tweets[requiredColumns]
clean_accounts = clean_tweets[requiredColumns]
def clean_df(df):
    
    df['retweet_count'] = df['retweet_count'].apply(lambda x: 1 if x==x else 0)
    df['reply_count'] = df['reply_count'].apply(lambda x: 1 if x==x else 0)
    df['favorite_count'] = df['favorite_count'].apply(lambda x: 1 if x==x else 0)
    df['num_hashtags'] = df['num_hashtags'].apply(lambda x: 1 if x==x else 0)
    df['num_urls'] = df['num_urls'].apply(lambda x: 1 if x==x else 0)
    df['num_mentions'] = df['num_mentions'].apply(lambda x: 1 if x==x else 0)
    #df['profile_use_background_image'] = df['profile_use_background_image'].apply(lambda x: 1 if x==x else 0)
    #df['verified']=df['verified'].apply(lambda x: 1 if x==x else 0)
    #df['is_protected']=df['protected'].apply(lambda x: 1 if x==x else 0)
    #df['default_profile'] = df['default_profile'].apply(lambda x: 0 if x==x else 1)
    #df = df.rename(index=str, columns={"screen_name": "username", "statuses_count": "total_tweets", "friends_count": "total_following", "followers_count": "total_followers", "favourites_count": "total_likes"})
    return df[['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls','num_mentions']]

bot_accounts = clean_df(bot_accounts)
clean_accounts = clean_df(clean_accounts)
#print(bot_accounts)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

In [0]:
bot_accounts['BotOrNot'] = 1
clean_accounts['BotOrNot'] = 0
#print(bot_accounts)
#print(clean_accounts)

combined_df = pd.concat([bot_accounts, clean_accounts])

new_df = combined_df.sample(frac=1).reset_index(drop=True)

In [0]:
training_df = new_df[:int(combined_df.shape[0] * 0.8)]
test_df = new_df[int(combined_df.shape[0] * 0.8):]



columns_to_standardize = ['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls','num_mentions']

training_df_mean = training_df[columns_to_standardize].mean()
training_df_std = training_df[columns_to_standardize].std()

#training_df[columns_to_standardize] = (training_df[columns_to_standardize] - training_df_mean)/training_df_std
#test_df[columns_to_standardize] = (test_df[columns_to_standardize] - training_df_mean)/training_df_std

# training_df_mean = training_df.mean()
# training_df_std = training_df.std()

# training_df = (training_df - training_df_mean)/training_df_std
# test_df = (test_df - training_df_mean)/training_df_std

# max_vals = training_df.max()

# training_df = training_df/max_vals
# test_df = test_df/max_vals


In [8]:
#X_train = training_df.drop(['BotOrNot', 'is_protected'], axis=1).values
#y_train = training_df['BotOrNot'].values.reshape(-1,1)
#print(X_train)
X_train = training_df
y_train = training_df['BotOrNot'].values.reshape(-1,1)

X_test = test_df
y_test = test_df['BotOrNot'].values.reshape(-1,1)
print()
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
sm = SMOTEENN()
X_train, y_train = sm.fit_sample(X_train, y_train)
#print(X_train)
from sklearn.ensemble import AdaBoostClassifier

# Import Support Vector Classifier
from sklearn.svm import SVC
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
svc=SVC(probability=True, kernel='linear')

# Create adaboost classifer object
abc =AdaBoostClassifier(n_estimators=50, base_estimator=svc,learning_rate=1)

# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)


# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 1.0


In [0]:
#predicted_classes = np.argmax(np.round(predicted_classes))
#print(classification_report(y_train, predicted_classes, target_names=target_names))
def get_accuracy(AL, y, verbose=1):
    
    try:
        AL = np.array(AL)
        y = np.array(y)

        AL = AL.reshape(-1)
        y = y.reshape(-1)

        AL = AL > 0.5
        AL = AL.astype(int)

        y = y > 0.5
        y = y.astype(int)

        total = AL.shape[0]

        TP = np.sum(np.logical_and(AL==1, y==1))
        TN = np.sum(np.logical_and(AL==0, y==0))

        FP = np.sum(np.logical_and(AL==1, y==0))
        FN = np.sum(np.logical_and(AL==0, y==1))

        P = TP / (TP + FP)
        R = TP / (TP + FN)
        F1 = (2 * P * R) / (P + R)


        acc = np.sum(AL == y)/total


        if verbose == 1:
            print("\nAccuracy: {} \n".format(acc))
            print("True Positive: {} \nTrue Negative: {}\nFalse Positive: {} \nFalse Negative: {}\n".format(TP, TN, FP, FN))
            print("Precision: {} \nRecall: {} \nF1 Score: {}\n".format(P, R, F1))
        
        return acc
    except:
        return 0

In [10]:
y_predict = abc.predict(X_train)
get_accuracy(y_predict, y_train)


Accuracy: 1.0 

True Positive: 160047 
True Negative: 160047
False Positive: 0 
False Negative: 0

Precision: 1.0 
Recall: 1.0 
F1 Score: 1.0



1.0

In [12]:
#y_test_predict = abc.predict(X_test)
get_accuracy(y_pred, y_test)


Accuracy: 1.0 

True Positive: 40047 
True Negative: 39953
False Positive: 0 
False Negative: 0

Precision: 1.0 
Recall: 1.0 
F1 Score: 1.0



1.0